## Scraping the visits of the US and Chinese presidents

In [1]:
#importing the libraries
import pandas as pd
%matplotlib inline
import requests
from bs4 import BeautifulSoup

# US visits

US presidential visits are annonced on this Office of the Historian webpage https://history.state.gov/departmenthistory/travels/president until December 2018. After that the page hasn't been updated but we can get all the visits before that. 


In [2]:
#the visits are in a table so I used the IMPORTHTML-function is sheets and scraped the tables of Bush, Obama and Trump with that and then combined them.
df2=pd.read_csv("../data/US STATE VISITS 2001-2018 - Sheet1.csv")

In [3]:
#t looks like this
df2.head()

,Country,Locale,Remarks,Date,Unnamed: 4
0,Mexico,San Cristobal,Meeting with President Fox.,"February 16, 2001",W. BUSH
1,Canada,Quebec,Attended the Summit of the Americas.,"April 20–22, 2001",W. BUSH
2,Spain,Madrid,Met with King Juan Carlos I and President Aznar.,"June 12–13, 2001",W. BUSH
3,Belgium,Brussels,Attended NATO Summit Meeting.,"June 13–14, 2001",W. BUSH
4,Sweden,Goteborg,Attended U.S.-European Union Summit Meeting; m...,"June 14–15, 2001",W. BUSH


Since the data finishes to December 2018 I then collected the rest of the event from the news sources and added to the frame by hand.

## Scraping China visits


The visits by Chinese presidents can be tracked via China Vitae http://www.chinavitae.com that tracks appearences and traveling of Chinese VIP's and annonces them. 
I use BeautiflSoup for scraping the trips

In [4]:
def scrape_visit_link(visit_link):
    r2 = requests.get(visit_link)
    soup2 = BeautifulSoup(r2.content, "html.parser")
    info_table = soup2.find_all("table")[1].find("table")
    row_dict = {}
    for row in info_table.find_all("tr"):
        category = row.find("b").string.strip().strip(":")
        if category == "Source":
            info = row.find("a")["href"]
        elif category == "Attendees":
            links = row.find_all('a')
            info = [ l.string for l in links ]
        else:
            info = row.find_all("td")[1].string
        row_dict[category] = info
    return row_dict

In [5]:
def scrape_china_visits(pres_link):
    visit_dicts = []
    r = requests.get(pres_link)
    soup = BeautifulSoup(r.content, "html.parser")
    all_links = soup.findAll("a")
    complete_links =  []
    for al in all_links:
        if al.string == "View complete entry":
            complete_links.append(al)
    for cl in complete_links:
        visit_link = "http://www.chinavitae.com/" + cl["href"]
        visit_dict = scrape_visit_link(visit_link)
        visit_dicts.append(visit_dict)
    return visit_dicts

In [8]:
all_visits = []
y = 2003
while y <= 2019:
    print(y)
    #President Hu Jintao
    if y < 2013:
        pres_id = 19 
    elif y <= 2019:
        #President Xi Jinping
        pres_id = 303
    pres_link = "http://www.chinavitae.com/vip/index.php?mode=events&type=cv&id={}&filter_year={}".format(pres_id, y)
    visits = scrape_china_visits(pres_link)
    all_visits.append(visits)
    y += 1
#president Jiang Zemin left power in March 2003 so his visits 2001 and 2002 have to be researched individually.    

2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [9]:
visits = [item for y in all_visits for item in y]
visits_df = pd.DataFrame(visits)

In [10]:
visits_df.head()

,Date,Activity,Location,Attendees,Topics,Source
0,"December 31, 2003","In his New Year's address, broadcast by China ...",Beijing Municipality,[Hu Jintao],domestic politics,NaN
1,"December 30, 2003",Hu Jintao and other senior leaders welcomed th...,Beijing Municipality,"[Hu Jintao, Cao Gangchuan, Chen Kuiyuan, Chen ...",domestic politics,NaN
2,"December 29, 2003",Hu Jintao met with the US president's special ...,Beijing Municipality,"[Hu Jintao, Li Zhaoxing, \n\t\t\t\t\tJames Baker]",foreign relations,NaN
3,"December 26, 2003",Hu Jintao visited the mausoleum of Mao Zedong ...,Beijing Municipality,[Hu Jintao],domestic politics,NaN
4,"December 26, 2003",Hu Jintao gave the keynote speech at the semin...,Beijing Municipality,"[Hu Jintao, Huang Ju, Jia Qinglin, Li Changchu...",domestic politics,NaN


In [11]:
visits_df.tail()

,Date,Activity,Location,Attendees,Topics,Source
5826,"January 02, 2019",Xi pledges utmost sincerity for peaceful reuni...,Beijing Municipality,[Xi Jinping],None,http://www.xinhuanet.com/english/2019-01/02/c_...
5827,"January 02, 2019",Xi expounds best approach to reunification,Beijing Municipality,[Xi Jinping],None,http://www.xinhuanet.com/english/2019-01/02/c_...
5828,"January 02, 2019",Xi proposes institutional arrangement for peac...,Beijing Municipality,[Xi Jinping],domestic politics,http://www.xinhuanet.com/english/2019-01/02/c_...
5829,"January 02, 2019",Xi highlights breakthroughs in cross-Strait re...,Beijing Municipality,[Xi Jinping],domestic politics,http://www.xinhuanet.com/english/2019-01/02/c_...
5830,"January 02, 2019",Xi delivers speech at gathering marking 40th a...,Beijing Municipality,[Xi Jinping],domestic politics,http://www.xinhuanet.com/english/2019-01/02/c_...


In [12]:
visits_df.to_csv("../data/all_china_visits.csv", index=None)

After this is done we have a table with all the appearences of current president. Appearences in China need to be filtered out. 
Also, the beginning of the year 2003 before 15 March has to be removed because they are from the time Jintao was the president.Same goes to the year 2013 when president changed. We need to have Jintao until 15 March and Jinping after that.

Also, there are often several entries on one trip so they need to be combined into one. I did this filtering and combining in Google sheets.



Since China Vitae only tracks since 2003, I collected the visits by Jiang Zemin 2001 and 2002 by hand via Chinese embassies and Xinhua pages. I added them to final database by hand. Source of the visit is included in every data entry.